# Auswertung - a

### README

Der Code visualisiert die Kombination vonT ypen innerhalb eines Märchenrepertoire.

Motive eines Typs wird durch das Attribut a dargestellt 1-Tupel (a=Typnummer).
Anpassbare Zeilen:

(#2 und #27) Pfad zu der Inputdatei.

(#6 und 30) Der Attributwert bestimmt die Herkunft der zu analysierenden Textgruppe (des Repertoires). Die Filter können mit den 'or' und 'and' Operatoren ergänzt werden, z. B.:  

if ('deu' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']

    or 'ita' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']): 
    
    

In [1]:
schema=""  #1 Eine leere Zeichenfolge wird initialisiert.
import xml.etree.ElementTree as ET
root_node = ET.parse('Textkorpus.xml').getroot()  #2 Die Wurzel des XML-Baums wird aus der Datei "Textkorpus.xml" gelesen und zugewiesen.
never_saved = True  #3 Eine boolsche Variable never_saved wird auf True gesetzt.
schema_dict = dict()  #4 Ein leeres Dictionary wird initialisiert.
for ganze in root_node.findall(".//{http://www.tei-c.org/ns/1.0}text"):  #5 Eine Schleife durchläuft alle Elemente mit dem Tag 'text'.
    if '' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']:  #6 Überprüft, ob ein leerer Wert in den Attributen vorhanden ist.
        for body in ganze.findall(".//{http://www.tei-c.org/ns/1.0}body"):  #7 Eine Schleife durchläuft alle Elemente mit dem Tag 'body'.
            for absatz in body.findall(".//{http://www.tei-c.org/ns/1.0}p"):  #8 Eine Schleife durchläuft alle Elemente mit dem Tag 'p'.
                for phrase in absatz.findall(".//{http://www.tei-c.org/ns/1.0}seg"):  #9 Eine Schleife durchläuft alle Elemente mit dem Tag 'seg'.
                    labela = phrase.attrib['{www.dglab.uni-jena.de/vmf/a}ana']  #10 Der Wert des Attributs '{www.dglab.uni-jena.de/vmf/a}ana' wird zugewiesen.
                    if labela.startswith('a'):  #11 Überprüft, ob der Wert des Attributs mit 'a' beginnt.
                        combined_label = labela  #12 Der Wert des Attributs wird zugewiesen.
                        if labela not in schema_dict:  #13 Überprüft, ob der Wert des Attributs nicht im Dictionary enthalten ist.
                            schema_dict[labela] = {combined_label}  #14 Ein neuer Eintrag wird im Dictionary erstellt.
                        else:
                            schema_dict[labela].add(combined_label)  #15 Der Wert wird dem entsprechenden Eintrag im Dictionary hinzugefügt.
for k, v in sorted(schema_dict.items()):  #16 Eine Schleife durchläuft die Elemente des sortierten Dictionarys.
    schema += f'graph.update(alle_aeste("{k}", df))\n'  #17 Ein String wird zur Variablen schema hinzugefügt.
with open("TypGraph.txt", 'w', encoding='utf-8') as f:  #18 Eine Datei mit dem Namen 'TypGraph.txt' wird im Schreibmodus geöffnet.
    f.write(schema)  #19 Der Inhalt der Variable schema wird in die Datei geschrieben.
#print(schema)  #20 Dieser Code ist auskommentiert.
def graph_update():  #21 Eine Funktion graph_update wird definiert.
    with open("TypGraph.txt", "r") as datei:  #22 Die Datei 'TypGraph.txt' wird im Lesemodus geöffnet.
        zusatz_code = datei.read()  #23 Der Inhalt der Datei wird in die Variable zusatz_code gelesen.
    exec(zusatz_code)  #24 Der Python-Code in zusatz_code wird ausgeführt.

In [2]:
maerchen = ""  #25 Eine leere Zeichenkette, in der Daten für Märchen gesammelt werden.
import xml.etree.ElementTree as ET  #26 Importieren der ElementTree-Bibliothek zum Parsen von XML.
root_node = ET.parse('Textkorpus.xml').getroot()  #27 Parsen der XML-Datei und Zugriff auf das Wurzelelement.
never_saved = True  #28 Ein boolean-Wert, der angibt, ob Daten bereits gespeichert wurden oder nicht.
#29 Iteration durch jedes 'text'-Element in der XML-Datei.
for ganze in root_node.findall(".//{http://www.tei-c.org/ns/1.0}text"):
    #30 Überprüfung, ob das Attribut 'deu' in der 'id'-Eigenschaft des 'text'-Elements vorhanden ist.
    if ('' in ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']):
        #31 Iteration durch jeden Absatz im 'body' des 'text'-Elements.
        for body in ganze.findall(".//{http://www.tei-c.org/ns/1.0}body"):
            for absatz in body.findall(".//{http://www.tei-c.org/ns/1.0}p"):
                #32 Iteration durch jede 'seg'-Phrase im Absatz.
                for phrase in absatz.findall(".//{http://www.tei-c.org/ns/1.0}seg"):
                    #33 Extrahieren von Labels und Inhalten aus den Attributen und dem Text der Phrase.
                    labela = phrase.attrib['{www.dglab.uni-jena.de/vmf/a}ana']
                    labelb1 = phrase.attrib['{www.dglab.uni-jena.de/vmf/b1}ana']
                    labelb2 = phrase.attrib['{www.dglab.uni-jena.de/vmf/b2}ana']
                    labelb3 = phrase.attrib['{www.dglab.uni-jena.de/vmf/b3}ana']
                    labelb4 = phrase.attrib['{www.dglab.uni-jena.de/vmf/b4}ana']
                    labelb5 = phrase.attrib['{www.dglab.uni-jena.de/vmf/b5}ana']
                    labelc1 = phrase.attrib['{www.dglab.uni-jena.de/vmf/c1}ana']
                    labelc2 = phrase.attrib['{www.dglab.uni-jena.de/vmf/c2}ana']
                    labelc3 = phrase.attrib['{www.dglab.uni-jena.de/vmf/c3}ana']
                    labelc4 = phrase.attrib['{www.dglab.uni-jena.de/vmf/c4}ana']
                    labelc5 = phrase.attrib['{www.dglab.uni-jena.de/vmf/c5}ana']
                    labeld = phrase.attrib['{www.dglab.uni-jena.de/vmf/d}ana']
                    #34 Die Quelle wird aus dem Attribut 'id' der 'ganze'-Schleife extrahiert.
                    quelle = ganze.attrib['{http://www.w3.org/XML/1998/namespace}id']
                    #35 Der Inhalt wird in Kleinbuchstaben umgewandelt und formatiert.
                    inhalt = (phrase.text.lower().strip().replace('|','').replace(':','').replace("ä","ae")
                              .replace("ü","ue").replace("ö","oe").replace("ß","ss").replace(",","")
                              .replace("«","").replace("»","").replace(".","").replace('=',' ').replace(";","")
                              .replace('"',"").replace("?","").replace("!","").replace("á","a").replace(",","")
                              .replace("\t"," ").replace("'","").replace("‹","").replace("›","").replace("-"," ")
                              .replace("'('","").replace("')'","").replace('>','').replace("    "," ")
                              .replace("   "," ").replace("  "," ").replace('–','').replace('—','').replace('<','')
                              .replace("Â", "A").replace("ø", "oe").replace('“','').replace('„','').replace('(','')
                              .replace(')','').replace('*','').replace("\n","\n")
                              .replace("'''",""))
                    #36 Überprüfung, ob labela mit 'a' beginnt und nicht gleich 'N' ist.
                    if labela.startswith('a') and labela != 'N':
                            #37 Konstruktion des Datensatzes für das Märchen und Anhängen an die 'maerchen'-Zeichenkette.
                            maerchen += (quelle+','+labela+','+labelb1+','+labelc1+','+labelb2+','+labelc2+','+
                                         labelb3+','+labelc3+','+labelb4+','+labelc4+','+labelb5+','+labelc5+','+
                                         labeld+','+inhalt+',0'+'\n')                       
with open("text.csv", 'w', encoding='utf-8') as f:
     #38 Schreiben der Kopfzeile in die CSV-Datei.
    f.write('quelle,labela,labelb1,labelc1,labelb2,labelc2,labelb3,labelc3,labelb4,labelc4,labelb5,labelc5,labeld,inhalt,index_binar')
    f.write('\n')
    #39 Schreiben der gesammelten Daten in die CSV-Datei.
    f.write(maerchen)

In [3]:
#40 leere Zeichenketten, in der Daten für abcd1-5 gesammelt werden.
abcd1 = ""
abcd2 = ""
abcd3 = ""
abcd4 = ""
abcd5 = ""
import pandas as pd
from collections import Counter
#41 Einlesen der CSV-Datei in ein DataFrame.
df = pd.read_csv('text.csv', encoding='utf-8')
#42 Hinzufügen von Daten an abcd1, abcd2, abcd3, abcd4 und abcd5.
abcd1 += df.quelle+','+df.labela+':'+df.labelb1+':'+df.labelc1+':'+df.labeld+','+df.inhalt+',0'
abcd2 += df.quelle+','+df.labela+':'+df.labelb2+':'+df.labelc2+':'+df.labeld+','+df.inhalt+',0'
abcd3 += df.quelle+','+df.labela+':'+df.labelb3+':'+df.labelc3+':'+df.labeld+','+df.inhalt+',0'
abcd4 += df.quelle+','+df.labela+':'+df.labelb4+':'+df.labelc4+':'+df.labeld+','+df.inhalt+',0'
abcd5 += df.quelle+','+df.labela+':'+df.labelb5+':'+df.labelc5+':'+df.labeld+','+df.inhalt+',0'
#43 Zusammenstellen der Daten aus abcd1 bis abcd5.
recorded = (abcd1 + '\n' + abcd2 + '\n' + abcd3 + '\n' + abcd4 + '\n' + abcd5 + '\n')
#44 Schreiben der aufgezeichneten Daten in eine CSV-Datei.
with open("text_aw.csv", 'w', encoding='utf-8') as w:
    w.write('quelle,index_string,inhalt,index_binar\n')
    w.write('000:ballast,ballast,ballast,0\n')  #45 Schreiben einer Platzhalterzeile.
    w.write("".join(recorded))  #46 Schreiben der aufgezeichneten Daten.
    w.close()
import csv
mom = []  #47 Eine leere Liste zum Speichern der Daten.

#48 Lesen der CSV-Datei und Filtern von Zeilen mit ':N:' in der zweiten Spalte.
with open('text_aw.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        if ':N:' not in line[1]:  #49 Wenn ':N:' nicht in der zweiten Spalte enthalten ist.
            mom.append(line)  #50 Die Zeile der Liste mom hinzufügen.
#51 Schreiben der gefilterten Daten in eine Textdatei.
with open('text_ax.txt', 'wt', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(mom)  # Schreiben der Liste in die Datei.
#52 Lesen der Textdatei und Formatieren der Daten.
with open('text_ax.txt', 'r', encoding='utf-8') as file:
    filedata = file.read()
filedata = (filedata.replace(']","[', "\n").replace('"[', '').replace(']"', '').replace(" '", "")
            .replace("'", ""))
#53 Schreiben der formatierten Daten in eine CSV-Datei.
with open('text_ay.csv', 'w', encoding='utf-8') as file:
    file.write(filedata)

In [4]:
import pandas as pd
from collections import Counter
#54 Lesen einer CSV-Datei mit dem Namen 'text_ay.csv' in ein DataFrame mit dem Namen df
df = pd.read_csv('text_ay.csv', encoding='utf-8')
#55 Definition einer Funktion, um ein spezifisches Format aus einem String zu extrahieren, der eine Episode repräsentiert
def ep_format(ep_full):
    return (ep_full.split(':'))[0]
#56 Definition einer Funktion, um den Namen einer Episode zu formatieren
def ep_name_format(ep_full):
    sublist = (ep_full).split(':')[0:1]  #57 Extrahieren des ersten Teils des Episoden-Strings
    name = ""
    for strg in sublist:
        name += strg  #58 Zusammensetzen der Teile des Episoden-Strings
    return name
#59 Definition einer Funktion, um die Quellen zweier Indizes im DataFrame zu vergleichen
def quellenvergleich(df, i1, i2):
    return df.quelle[i1] == df.quelle[i2]
#60 Definition einer Funktion, um Segmente (Episoden) im DataFrame zu analysieren
def ast(gesep, df):
    ep_tree = {}  #61 Initialisierung eines leeren Dictionarys zur Speicherung von Episodeninformationen
    a_liste = []  #62 Initialisierung einer leeren Liste zur Speicherung von Startpunkten der Segmente
    z_liste = []  #63 Initialisierung einer leeren Liste zur Speicherung von Endpunkten der Segmente
    df_len = len(df.index_string)  #64 Ermitteln der Länge der Spalte 'index_string' im DataFrame
    #65 Schleife durch den DataFrame, um Segmente zu analysieren
    for i, ep in enumerate(df.index_string):
        if gesep == ep:  #66 Überprüfen, ob die aktuelle Episode der angegebenen entspricht
            #67 Bestimmung des Startpunkts des Segments
            if (i > 0) and (quellenvergleich(df, i, i - 1)):
                a = df.index_string[i - 1]
            else:
                a = 'Anf-' + ep_name_format(gesep)
            #68 Bestimmung des Endpunkts des Segments
            if (i < df_len - 1):
                if not (quellenvergleich(df, i, i + 1)):
                    z = 'End-' + ep_name_format(gesep)
                else:
                    z = df.index_string[i + 1]
            else:
                z = 'End-' + ep_name_format(gesep)
            #69 Hinzufügen der Start- und Endpunkte zu ihren jeweiligen Listen
            a_liste.append(a)
            z_liste.append(z)
    #70 Konstruktion eines Dictionarys, das die Start- und Endpunkte des Segments enthält
    return {gesep: [Counter(a_liste), Counter(z_liste)]}

In [5]:
def alle_aeste(gesep, df):
    episoden_baeume = {}  #71 Ein leeres Dictionary zur Speicherung der Analyseergebnisse.
    ep_list = []  #72 Eine leere Liste zur Speicherung von Episoden.
    for ep_full in df.index_string:  #73 Iteration durch die 'Index-String' des DataFrames.
        ep = ep_format(ep_full)  #74 Formatierung der Episode.
        if gesep == ep:  #75 Wenn die gesuchte Episode gefunden wird.
            ep_list.append(ep_full)  #76 Die Episode wird zur Liste hinzugefügt.
    for ep in set(ep_list):  #77 Iteration durch jede einzigartige Episode in der Liste.
        episoden_baeume.update(ast(ep, df))  #78 Die Analyseergebnisse für jede Episode werden dem Dictionary hinzugefügt.
    return episoden_baeume  #79 Rückgabe des Dictionary mit den Analyseergebnissen.

In [6]:
#80 Die Variable 'graph' wird mit den Analyseergebnissen für die Episode 'mZahl' initialisiert.
graph = alle_aeste('', df)
#81 Die Funktion graph_update wird aktiviert
graph_update()

In [7]:
import matplotlib.pyplot as plt 
import networkx as nx 
#82 Eine Funktion zum Ausschneiden von Astdaten basierend auf einem Schwellenwert
def cutoff_ast_data(wuerzel, graph, nachbar_liste, cutoff=0):
    vor = graph[wuerzel][0]
    zurueck = graph[wuerzel][1]
    nachbarn = vor.copy()
    nachbarn.update(zurueck)
    for ast_name, ast_gewicht in sorted(nachbarn.items()):
        if (ast_gewicht >= cutoff):
             nachbar_liste.append([ep_name_format(wuerzel), 
                                   ep_name_format(ast_name), ast_gewicht])  
#83 Eine Funktion zum Ausschneiden von Graphdaten basierend auf einem Schwellenwert
def cutoff_graph_data(graph, cutoff=0):
    nachbar_liste = []
    for i, (k, v) in enumerate(sorted(graph.items())):
        cutoff_ast_data(k, graph, nachbar_liste, cutoff)
    neue_nachbar_liste = [[i, nachbar] for i, nachbar in enumerate(nachbar_liste)]
    return neue_nachbar_liste
#84 Eine Funktion zur Interaktion mit Graphdaten basierend auf einem Löschindex und einem Gewichts-Dictionary
def interactive_graph_data(graph_data, loesch_index, gew_dict):
    for k, v in gew_dict.items():
        graph_data[k][1][2] = v
    neue_nachbar_liste = [[i, nachbar[1]] for i, 
                          nachbar in enumerate(graph_data) if i not in loesch_index]
    return neue_nachbar_liste
#85 Eine Funktion zum Erstellen eines Graphen aus Graphdaten
def graph_bauer(graph_data):
    G = nx.DiGraph()  #86 Ein gerichteter Graph wird erstellt
    w_liste = []  #87 Eine Liste für die Gewichte wird initialisiert
    for el in graph_data:
        n1 = el[1][0]
        n2 = el[1][1]
        w = el[1][2]
        w_liste.append(w)
        G.add_edge(n1, n2, weight=w, title=w)
    return G, w_liste

In [8]:
auto_graph=cutoff_graph_data(graph)

In [9]:
from pyvis.network import Network 
#88 Erstellen des Graphen und der Gewichtsliste mithilfe der Funktion graph_bauer
G, W = graph_bauer(auto_graph)
#89 Initialisieren des Netzwerk-Objekts mit der Option 'notebook=True' für die Verwendung in einem Jupyter Notebook
nt = Network(notebook=True)
#90 Konvertieren des NetworkX-Graphen in ein pyvis-Netzwerk
nt.from_nx(G)
#91 Anzeigen des Netzwerkdiagramms und Speichern als HTML-Datei mit dem Namen 'nx.html'
nt.show('nx.html')

nx.html
